In [ ]:
# 태진님의 커널 필사 공부!
# 2019-07-05 
# Generator, keras Generator

# 이론 정리
# 개와 고양이를 분류하는 것과 달리, 차 종 분류는 일반적인 이미지 분류보다 어려움이 존재
# 분류 문제에서 가장 먼저 확인 -> target variables 의 distribution 확인
# Bounding Box : 이미지 내부에서 특정 object를 박스로 레이블한 것을 말함. 보통 좌측 상단, 우측 하단 좌표가 주어짐.
# -> 이러한 좌표를 찾는 모델도 있음! 
# -> 쓰는 이유, 어떤 이미지에는 내가 원하는 Target Object 뿐만 아니라, Noise가 섞여 있을 수 있음
#    그래서 Bounding Box 수행!
# -> 원래는 Bounding Box를 구하는 모델링을 수행해야 한다!


# trainSet / TestSet
# trainSet을 다시 vaildationSet 으로 나눔
# 보통 CV, LB라는 용어가 나오는데, CV : crossValidation 을 통한 검증 accuracy, LB : 리더보드에 올라간 Public Score를 의미

# ** 제너레이터(Generator)
# 배치사이즈 단위 만큼 파일을 불러와 학습하고 끝나면 다시 불러와서 학습하는 방법을 반복하는 것을 의미
# 전체 학습으 하더라도 메모리를 적게 사용! -> 핵심
# 코랩이나 캐글 커널 같은 클라우드 환경 OR 로컬 환경에서 정말 유용하게 사용할 수 있음
# 이미지 프로세싱에서 없어서는 안 될 필수 과정
 
# ** 케라스 제너레이터    
# ImageDataGenerator 는 제너레이터 기능은 물론, 제너레이터를 정의하면서, 동시에 Data에 원하는 Noise까지 부여
# 주어진 데이터가 이미지가 아닌 경우 직접 제너레이터를 설계해야 하는 경우도 존재.

# ** ResNet 
# 유명하다.
# Residual를 이용한 획기적인 모델로 평가됨
# Pretrained Model을 불러오기 위해서는 커널의 Internet 옵션이 활성화 되어 있어야 함

# **Pretrained(선행 학습, 사전 훈련, 전처리과정이라고 함) Model
# Milti Layered Perceptron 에서 Weight와 Bias를 잘 초기화 시키는 방법이다.
# 사용시 한가지 주의할 점. Pretrained Model은 경우에 따라 다양하게 사용될 수 있기 때문에, Model output 부분을 잘라버린 채 로드되는 경우가 있음.(include_top = False)
# 이럴 때는 직접 output을 만들어주어야 함

# macro 평균 vs micro 평균
#        A반    B반    C반
#  학생수  9     10     8
#  평균   40     70    90

# 평균의 합을 반의 개수로 나누는 방법 -> macro 평균
# 전체의 값들의 평균 개념 -> Micro 평균

# library load
import gc             # garbage collection library
import os             # 기본적인 환경을 setting 할 수 있는 library(ex) dir 등) 
import warnings       # warnings을 무시하기 위해 일반적으로 사용되는 library
import numpy as np    # numpy package : array, 배열 연산, slicing 에 자주 사용되는 library
import pandas as pd   # 주로 분석 모델에 사용하는 package
import seaborn as sns # matplotlib를 support 해주는 library
import matplotlib.pyplot as plt 
from tqdm import tqdm # 진행바를 만들어주는 library

from keras import backend as K # 딥러닝 모델 변수들의 초기값을 setting해 주기 위한 library
warnings.filterwarnings(action = 'ignore')
K.image_data_format() # channel_first 인 경우, (샘플 수, 랭크(축) 수, 행, 열)  4D tensor 필터 수 ->  축 수
                      # channel_last  인 경우, (샘플 수, 행, 열, 필터 수)      4D tensor  -> default

DATA_PATH = '../input/'  # 기본 디렉토리 설정하기 위한 변수 선언
os.listdir(DATA_PATH)    # 기본 디렉토리 설정 -> 해당 디렉토리의 file 이나 folder 확인 가능

# 이미지 폴더 경로
TRAIN_IMG_PATH = os.path.join(DATA_PATH, 'train')
TEST_IMG_PATH = os.path.join(DATA_PATH, 'test')

# CSV 파일 경로를 통해 data read : train, test, class
df_train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
df_test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
df_class = pd.read_csv(os.path.join(DATA_PATH, 'class.csv'))

df_train.head() # 몇개만 데이터 확인해보기 위해, head() 사용
df_test.head()  # test data 이기 때문에, class가 없는 것을 확인!

# 데이터 누락 check
# .csv 에 있는 list와, 실제 data list의 개수가 맞는지 비교 확인
# dataframe.colname 하면 해당 값이 나온다.
if set(list(df_train.img_file)) == set(os.listdir(TRAIN_IMG_PATH)) :
    print("Train file 누락 없음!")
else:
    print("Train file 누락!")

if set(list(df_test.img_file)) == set(os.listdir(TEST_IMG_PATH)) :
    print("Test file 누락 없음!")
else:
    print("Test file 누락!")

# Data 개수 확인!
# train : 9990
# test  : 6150
print("Number of Train Data : {}".format(df_train.shape[0]))
print("Number of Test  Data : {}".format(df_test.shape[0]))

# 클래스 개수 및 target vriable class 개수 확인 
# 실제로, 매핑되지 않을 수도 있음
# -> 196개로 동일 !
print("총 클래스 개수 : {}".format(df_class.shape[0]))
print("총 클래스 개수 : {}".format(df_train['class'].nunique()))


# Class Distribution

# seaborn library 이용.
# 그리고자 할 data , order 를 주고싶은 경우, 해당 index 입력!(아마도..)
plt.figure(figsize = (12, 6)) # img size setting
sns.countplot(df_train["class"], order = df_train["class"].value_counts(ascending = True).index) # countplot(x="column_name", data=dataframe)
plt.title("Number of data per each class")
plt.show()

# -> plot 해석
# 대부분 class 의 개수가 고르게 분포되어 있음을 확인

# class 개수의 min, max 확인
cntEachClass = df_train["class"].value_counts(ascending=False)
print("최대 class 개수 : {}".format(cntEachClass.max()))
print("최소 class 개수 : {}".format(cntEachClass.min()))

cntEachClass.describe() # describe 함수를 통한 요약 정보 파악

# 파이썬에서 이미지 보고 싶은 경우,
# PIL library ->  이미지 로드하는 라이브러리라고 생각하자

import PIL
from PIL import ImageDraw

tmp_imgs = df_train['img_file'][100:110] # 이미지 10개 사용
plt.figure(figsize = (12, 20))           # 사진 size 설정
for num, f_name in enumerate(tmp_imgs):  
    img = PIL.Image.open(os.path.join(TRAIN_IMG_PATH, f_name)) # PIL library 의 image open 함수 이용 PATH, NAME 필요
    plt.subplot(5, 2, num + 1) # 5행 1열중, num + 1 째 위치(?)
    plt.title(f_name)          # Image title 설정
    plt.imshow(img)            # show
    plt.axis('off')            # 축은 안보이게 off

# Bounding Box
# 이미지 내부에서 특정 object를 박스로 레이블한 좌표
# 좌측 상단, 우측 하단 좌표가 주어진다
# 좌표는 이미지의 픽셀 좌표 

# draw_rect
# 이미지의 Bounding Box 를 그려주는 함수 
# @param drawcontext -> 그리고자 하는 img 객체(ImageDraw.Draw(img))
# @param pos,        -> [x1, y1, x2, y2]
# @param outline     -> 선의 색깔
# @param width       ->  선의 폭
def draw_rect(drawcontext, pos, outline = None, width = 0) : #
    (x1, y1) = (pos[0], pos[1]) # x1, y1 입력
    (x2, y2) = (pos[2], pos[3]) # x2, y2 입력
    points   =  (x1, y1), (x2, y1), (x2, y2), (x1, y2), (x1, y1) # 선을 4개 긋기 위한 좌표 5개. 다시 (x1, y1) 으로 회귀
    drawcontext.line(points, fill = outline, width = width) # drawcontext.line function

# make_boxing_img function
# bounding box를 적용한 이미지 출력 함수.
# 기존 좌표로 주어진, (x1, y1, x2, y2) 의 bounding axis가 제대로 되어 있는지? 함 확인하고자 만든 함수.
# @param img_name -> 이미지 이름
def make_boxing_img(img_name):
    # img가 train, test 인지 여부에 따라 PATH, data 각각 저장
    if img_name.split('_')[0] == "train" :
          PATH = TRAIN_IMG_PATH
          data = df_train
    elif img_name.split('_')[0] == "test" :
        
        PATH = TEST_IMG_PATH
        data = df_test
    
    # 선이 그려지지 않은 이미지 한개를 open 할 객체를 img에 저장
    img = PIL.Image.open(os.path.join(PATH, img_name))
    # reshape 함수는 Python을 통해 머신러닝 혹은 딥러닝 코딩을 하다보면 꼭 나오는 numpy 내장 함수입니다.
    #  reshape(-1) 인 경우 -> 구조화 되어 있는 dataframe을 1차원 배열을 반환  
    pos = data.loc[data["img_file"] == img_name, ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2']].values.reshape(-1) # values : header를 제거하고 값만 가져오는 함수.
    draw = ImageDraw.Draw(img)                        # 이미지를 그리고,
    draw_rect(draw, pos, outline = 'red', width = 10) # Bounding Box를 생성
        
    return img

f_name = "train_00102.jpg" # 이미지 샘플 1개 이름 저장
plt.figure(figsize = (20, 10)) # 이미지 크기 지정 
plt.subplot(1, 2, 1)

# Original Image
origin_img = PIL.Image.open(os.path.join(TRAIN_IMG_PATH, f_name))
plt.title("Original Image - {}".format(f_name))
plt.imshow(origin_img)
plt.axis('off')

# Image included bounding box
plt.subplot(1, 2, 2)
boxing = make_boxing_img(f_name)
plt.title("Boxing Image - {}".format(f_name))
plt.imshow(boxing)
plt.axis('off')

plt.show()

# 분류 모델 만들기
# ResNet50 Pretrained Model 불러와 사용해보자

# Train_test_split : Train 과 Test 를 split(?)
from sklearn.model_selection import train_test_split # train_test_split function 호출
df_train["class"] = df_train["class"].astype('str')  # class column data -> str로 변경

# 이번 모델링에서 Bounding Box를 쓰지 않기때문에, 해당 컬럼은 빼고 다시 저장
df_train = df_train[['img_file', 'class']] 
df_test  = df_test[['img_file']]

its = np.arange(df_train.shape[0]) # arange -> [0:10016]
train_idx, val_idx = train_test_split(its, train_size = 0.8, random_state=42) # 해당 idx 번호를 8:2 로 데이터 셋 분할(CV 생성)

X_train = df_train.iloc[train_idx, :]
X_val   = df_train.iloc[val_idx, :]

print(X_train.shape)
print(X_val.shape)
print(df_test.shape)

# Keras Generator 사용

from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing.image import ImageDataGenerator

# parameter
img_size = (224, 224) # 225, 225개의 행, 열
nb_train_samples = len(X_train)    # 샘플 수(train data 개수) 
nb_validation_samples = len(X_val) # 샘플 수(valiadation data 개수)
nb_test_samples = len(df_test)     # 샘플 수(test data 개수)

epochs = 20     # 학습용 사진 전체를 한 번 사용했을 때 한 세대(이폭, epoch)이 지나갔다고 말함. 즉, 전체 train Dataset 학습을 총 20번 하겠다는 의미
batch_size = 32 # batch size: 한 번에 처리하는 사진의 장 수

# Define Generator config 
# 1.train_datagen
train_datagen = ImageDataGenerator(
    horizontal_flip = True,  # 좌우 뒤집기 -> 좌우 반전을 통해 데이터를 증가시키고, 모델 향상 시킬 수 있음
    vertical_flip   = False, # 상하 뒤집기 -> 
    zoom_range      = 0.10,  # 실수 또는 [하한, 상한], 무작위 줌을 위한 범위
    preprocessing_function = preprocess_input # vgg16용 preprocess_input을 사용.
)

# 2. val_datagen, test_datagen
# train_datagen 과는 달리, 다른 parameter 값을 setting 하지 않음
val_datagen  = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Make Generator
# 해당 parameter 는 keras 책을 더 공부하고 알 수 있을듯 하다

train_generator = train_datagen.flow_from_dataframe(

    dataframe =  X_train,            # 실제 데이터의 img.name / class 존재하는 dataframe
    directory = '../input/train/',   # 해당 데이터의 존재 directory
    x_col     = 'img_file',          # x -> img_file 
    y_col     = 'class',             # y -> class
    target_size = img_size,          # img_size -> (225, 225)
    color_mode = 'rgb',
    class_mode = 'categorical',      # 196개의 class -> categorical_entropy. 2진 분류시 crossentropy..
    batch_size = batch_size,         # 32
    seed = 42                        # 계속 동일한 값을 가져오기 위해 seed 값 지정
    
)

# 2. validation_generator 
validation_generator = val_datagen.flow_from_dataframe(
    
    dataframe   = X_val,             
    directory   = '../input/train',
    x_col       = 'img_file',
    y_col       = 'class',
    target_size = img_size,
    color_mode  = 'rgb',
    class_mode  = 'categorical',
    batch_size  = batch_size,
    shuffle     = False
    
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe   = df_test,
    directory   = '../input/test',
    x_col       = 'img_file',
    y_col       = None,
    target_size = img_size,
    color_mode  = 'rgb',
    class_mode  = None,
    batch_size  = batch_size,
    shuffle     = False
)


# Pretained Model을  불러오기 위해서는 커널의 Internet 옵션이 활성화 되어 있어야 함
resNet_model = ResNet50(
    include_top = False,         # 최상단의 softmax 레이어를 제거
    input_shape = (224,224,3)    # input Tensor -> 224, 224, 3
)

# ResNet50을 실제로 만들어보자

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, GlobalAveragePooling2D

model = Sequential()     # model 객체 생성
model.add(resNet_model)  # resNet_model frame 추가
model.add(GlobalAveragePooling2D()) # 
model.add(Dense(196, activation = 'softmax', kernel_initializer = 'he_normal')) # 최종 output layer. 가중치 초기화 -> he_normal 방식 이용

model.summary()
# Model Compile 
# 이제 모델을 만들었으니, 어떻게 학습할지를 정해야 함.
# 어떤 방법으로, 어떤 속도로, 어떤 지표를 기준으로 등등 정할 수 있음

from sklearn.metrics import f1_score # sklearn에 f1_score 함수 호출

def micro_f1(y_true, y_pred):
    return f1_score(y_true, y_pred, average = 'micro')

model.compile(
    optimizer='adam',                       # optimizer -> adam..? , 케라스 예제에서는 rmsprop을 쓰던데..
    loss = 'categorical_crossentropy',      # categorical_crossentropy
    metrics = ['acc']                       # 정확도 계산..?
)

# Model Training
# step_for_epoch를 구하기 위한 함수인듯.
# 나머지가  0이면, 나누어 떨어진다는 뜻이므로, // .
# 나머지가 >0이면, //에 +1
def get_steps(num_samples, batch_size):
    if (num_samples % batch_size) > 0:
        return (num_samples // batch_size) + 1
    else:
        return num_samples // batch_size

%%time
# ModelCheckpoint -> callback함수로, 매 epoch 마다 학습된 가중치를 파일로 저장할 수 있음.
# EarlyStopping   -> 이전 epoch 때와 비교해서 오차가 증가했다면 학습을 중단.
from keras.callbacks import ModelCheckpoint, EarlyStopping

# model명을 acc, loss 명으로 해서 저장!
# -> good idea 인듯!
filepath = "my_resnet_model_{val_acc:.2f}_{val_loss:.4f}.h5"

es = EarlyStopping(monitor='val_acc', # 관찰하고자 하는 항목
                   min_delta =0, # 개선되고 있다고 판단하기 위한 최소 변화량
                   patience = 3, # 개선이 없는 에포크를 얼마나 기다려 줄 것인 가를 지정
                   verbose = 1,  # 얼마나 자세하게 정보를 표시할 것인가를 지정합니다. (0, 1, 2)
                   mode = 'auto' # 관찰 항목에 대해 개선이 없다고 판단하기 위한 기준을 지정
                                 # auto : 관찰하는 이름에 따라 자동으로 지정
                                 # min  : 관찰하고 있는 항목이 감소되는 것을 멈출 때 종료
                                 # max  : 관찰하고 있는 항목이 증가되는 것을 멈출 때 종료
                  )
    
# model fit generator에 callback param에 list형태로 들어가야 하므로, list 로 저장
callbackList = [es] 

# 모델 학습.
# 꽤 오랜시간이 걸리므로, 자기 전에 돌리고 자자^^ 
history = model.fit_generator(

    train_generator,
    steps_per_epoch  = get_steps(nb_train_samples, batch_size),
    epochs           = epochs,
    validation_data  = validation_generator,
    validation_steps = get_steps(nb_validation_samples, batch_size), 
    callbacks        = callbackList
)

gc.collect()

# Training History Visualization
# 학습된 결과를 plot으로 그려볼 수 있음.
# 

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Predict & Make submission
# Model predict
%%time
test_generator.reset()
prediction = model.predict_gerator(

    generator = test_generator,
    steps     = get_steps(nb_test_samples, batch_size),
    verbose   = 1 
)

# 중요!
# 케라스 제너레이터를 사용하는 경우에는 타겟(클래스)의 카테고리컬 매핑이
# 제너레이터 임의로 결정.
# 따라서 제너레이터가 가지고 있는 class index 딕셔너리 불러와 새롭게 매핑해주어야 함.

predicted_class_indices = np.argmax(prediction, axis = 1)

# Generator class dictionary mapping
labels      = (train_generator.class_indices)
labels      = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

submission          = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))
submission["class"] = predictions
submission.to_csv("submission.csv", index=False)
submission.head()






    
    